# Projeto de Séries Temporais com Regressão Linear

## 1. Contexto


---

**O presente** dataset faz parte de uma competição do [Kaggle](https://www.kaggle.com/c/competitive-data-science-predict-future-sales/overview) do Coursera, [Como Vencer uma Competição de Ciência de Dados](https://www.coursera.org/learn/competitive-data-science/home/welcome). A temática da competição é trabalhar em um projeto de Séries Temporais (TS) com frequência diária de vendas, disponibilizada por uma empresa russa chamada [1C Company](https://1c.ru/eng/title.htm).


**O objetivo** é prever o total de vendas de todos os produtos e história para o próximo mês, levando em consideração o tópico de TS, então temos que fazer essa previsão para 30 períodos a frente. Com isso, temos os seguintes banco de dados e as suas descrições:


a) **sales_train.csv:** Dados históricos de vendas, de Janeiro de 2013 a Outubro 2015

b) **test.csv:** Dados de teste, Novembro de 2015 (os 30 períodos a frente para prever)

c) **items.csv:** Informações adicionais de itens/produtos

d) **item_categories.csv:** Informações adicionais sobre as categorias de item

e) **shops:** Informações adicionais sobre as lojas


**Com** essas informações, criamos um notebook auxiliar para não deixar este relativamente poluído, criando funções, tratando dados, etc. E para isso, utilizamos o comando mágico do jupyter `%run`, para importar os objetos (bancos de dados, funções, etc).

In [1]:
%run function_ts_regressao.ipynb

## 2. Descrição dos dados

---

**Como** mencionado anteriormente, possuem dados em diferentes bancos de dados, podemos ver uma descrição um pouco mais aprofundada de cada um destes. Não iremos utilizar todos os bancos de dados, somente os dados de **sales_train, items, item_categories e shops**.

### 2.1. Sales Train 

i) **date**: data no formato dd/mm/yyyy;

ii) **date_block_num**: Mês consecutivo desde o início até o fim, isto é, 0 corresponde à Janeiro de 2013 e 33 a Outubro de 2015;

iii) **shop_id**: Identificador do shop;

iv) **item_id**: Identificador do item;

v) **item_price**: Preço atual do item;

vi) **item_cnt_day**: Número de produtos vendidos (**target**)

In [9]:
sales_train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [10]:
sales_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2935849 entries, 0 to 2935848
Data columns (total 6 columns):
 #   Column          Dtype  
---  ------          -----  
 0   date            object 
 1   date_block_num  int64  
 2   shop_id         int64  
 3   item_id         int64  
 4   item_price      float64
 5   item_cnt_day    float64
dtypes: float64(2), int64(3), object(1)
memory usage: 134.4+ MB


### 2.2. Items

i) **item_name**: Nome do item

ii) **item_id**: Identificador do item

iii) **item_category_id**: Identificador único da categoria do item

In [13]:
items.head()

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


In [14]:
items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22170 entries, 0 to 22169
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   item_name         22170 non-null  object
 1   item_id           22170 non-null  int64 
 2   item_category_id  22170 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 519.7+ KB


### 2.3. Item categories

i) **item_category_name**: Nome da categoria do item

ii) **item_category_id**: Identificador da categoria do item

In [16]:
item_categories.head()

,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


In [17]:
item_categories.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84 entries, 0 to 83
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   item_category_name  84 non-null     object
 1   item_category_id    84 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 1.4+ KB


### 2.4. Shops

i) **shop_name**: Nome da loja

ii) **shop_id**: ID da Loja 

In [19]:
shops.head()

,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4


In [20]:
shops.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   shop_name  60 non-null     object
 1   shop_id    60 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 1.1+ KB


### 2.5. Conclusões

Temos que fazer algumas modificações vendo os dados, temos que fazer algumas modificações nos tipos de dados, criação de features, etc.